In [11]:
import sys,os
import torch
current_directory = os.getcwd()
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname(os.path.abspath(os.path.dirname(current_directory))))))
from datasets.weather_bench import WeatherDataset

In [12]:
device = ("cuda" if torch.cuda.is_available() else "cpu" )
device = torch.device(device)

weather = WeatherDataset(0, device=device)
# dataset.shape:  torch.Size([7309, 100, 1450])
original = weather.load()

데이터셋 불러오는 중...


==== LOAD DATASET ====
 <xarray.Dataset>
Dimensions:                   (time: 7305, latitude: 28, longitude: 28,
                               level: 13)
Coordinates:
  * latitude                  (latitude) float32 39.0 38.75 38.5 ... 32.5 32.25
  * level                     (level) int64 50 100 150 200 ... 700 850 925 1000
  * longitude                 (longitude) float32 124.2 124.5 ... 130.8 131.0
  * time                      (time) datetime64[ns] 2016-12-31 ... 2021-12-31
Data variables: (12/14)
    10m_u_component_of_wind   (time, latitude, longitude) float32 ...
    10m_v_component_of_wind   (time, latitude, longitude) float32 ...
    2m_temperature            (time, latitude, longitude) float32 ...
    geopotential              (time, level, latitude, longitude) float32 ...
    mean_sea_level_pressure   (time, latitude, longitude) float32 ...
    sea_surface_temperature   (time, latitude, longitude) float32 ...
    ...                        ...
    total_cloud_cover         

Processing futures: 4it [00:03,  1.30it/s]

sea_surface_temperature


Processing futures: 14it [00:12,  1.15it/s]
Processing futures: 2it [00:00,  4.30it/s]


...

12.15630 sec
==== LOAD DATASET ====
 <xarray.Dataset>
Dimensions:                   (time: 7305, longitude: 17, latitude: 12,
                               level: 13)
Coordinates:
  * latitude                  (latitude) float64 2.903 8.71 ... 60.97 66.77
  * level                     (level) int64 50 100 150 200 ... 700 850 925 1000
  * longitude                 (longitude) float64 90.0 95.62 ... 174.4 180.0
  * time                      (time) datetime64[ns] 2016-12-31 ... 2021-12-31
Data variables: (12/14)
    10m_u_component_of_wind   (time, longitude, latitude) float32 ...
    10m_v_component_of_wind   (time, longitude, latitude) float32 ...
    2m_temperature            (time, longitude, latitude) float32 ...
    geopotential              (time, level, longitude, latitude) float32 ...
    mean_sea_level_pressure   (time, longitude, latitude) float32 ...
    sea_surface_temperature   (time, longitude, latitude) float32 ...
    ...                        ...
    total_cloud_cover 

Processing futures: 5it [00:02,  2.35it/s]

sea_surface_temperature


Processing futures: 14it [00:10,  1.37it/s]
Processing futures: 2it [00:00, 15.94it/s]


10.47663 sec
======= RESULT SHAPE =======
result_dataset.shape:  torch.Size([7305, 100, 1450])


In [15]:
%load_ext autoreload
%autoreload 2
from models.VariableAnalyzer.datasets.dataset import CustomDataset
dataset = CustomDataset(original, 4 * 7)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
print(dataset[0])

0


TypeError: list indices must be integers or slices, not list